## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7bfb2da8f1d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7bfb2da8fc80>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
# ai_result = model.invoke([HumanMessage(content="Hi , My name is Thang and I am an AI Engineer")])

In [4]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

### Using LCEL- chain the components
chain=model|parser
chained_result = chain.invoke([HumanMessage(content="Hi , My name is Thang and I am an AI Engineer")])
chained_result

"Hello Thang,\n\nIt's nice to meet you! As an AI, I don't have a name, but you can call me Gemma.\n\nWhat can I do for you today? Are you working on any interesting projects?\n"

In [26]:
## Hardcoded the previous response into AIMessage and try asking for history
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Thang and I am an AI Engineer"),
        AIMessage(content=chained_result),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Thang, and that you are an AI Engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 94, 'total_tokens': 136, 'completion_time': 0.076363636, 'prompt_time': 0.002715823, 'queue_time': 0.004044987, 'total_time': 0.079079459}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1e954457-d964-4944-a762-41e72d5726c8-0', usage_metadata={'input_tokens': 94, 'output_tokens': 42, 'total_tokens': 136})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
%pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [9]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Thang and I am an AI Engineer")],
    config=config
)

In [10]:
response.content

"Hello Thang! It's nice to meet you. \n\nThat's a fantastic field to be in. What kind of AI engineering work do you do? \n\nI'm always interested to learn more about how people are using AI to make a difference.\n"

In [11]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='You told me your name is Thang!  😊  \n\nIs there something else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 94, 'total_tokens': 120, 'completion_time': 0.047272727, 'prompt_time': 0.002546425, 'queue_time': 0.003840364, 'total_time': 0.049819152}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-06e3c48d-55d1-4312-b1cf-f1b9b0817d8a-0', usage_metadata={'input_tokens': 94, 'output_tokens': 26, 'total_tokens': 120})

In [12]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to remember it for this conversation.\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you! 👋 \n\nWhat can I do for you today?\n"

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John. 😊  \n\nIs there anything else I can help you with, John?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        # Using messages attribute to remember the previous messages
        MessagesPlaceholder(variable_name="messages")
    ]
)

# Piping output of prmopt to model
chain=prompt|model

In [16]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Thang")]})

AIMessage(content="Hello Thang! It's nice to meet you.\n\nI'm happy to help with any questions you have. Just ask away! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 31, 'total_tokens': 66, 'completion_time': 0.063636364, 'prompt_time': 0.000318399, 'queue_time': 0.00336128, 'total_time': 0.063954763}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1f6ba526-f044-4a93-ab28-21ea8941b803-0', usage_metadata={'input_tokens': 31, 'output_tokens': 35, 'total_tokens': 66})

In [17]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [18]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Thang")],
    config=config
)

response

AIMessage(content="Hello Thang, it's nice to meet you! \n\nI'm happy to help with any questions you have. What can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 31, 'total_tokens': 71, 'completion_time': 0.072727273, 'prompt_time': 0.00031887, 'queue_time': 0.0034232399999999997, 'total_time': 0.073046143}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4e42f789-99d2-4b53-8eef-f44b1336fdb9-0', usage_metadata={'input_tokens': 31, 'output_tokens': 40, 'total_tokens': 71})

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Thang!  I remember that.  😄 \n\n\n\n'

In [20]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [21]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Thang")],"language":"Vietnamese"})
response.content

'Chào Thang! Rất vui được gặp bạn. \n\nBạn có muốn hỏi tôi điều gì không? Tôi sẽ cố gắng hết sức để trả lời bạn. 😊\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [22]:
'''This one-liner sets a variable called input_messages_key to the string "messages." Typically, this key name is used in a dictionary or similar structure to reference the user’s messages within a conversation flow.

By defining a custom key, the code can later retrieve or store chat messages under the label "messages," making the data handling more readable and consistent in a multi-step conversation pipeline.'''
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [23]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Thang")],"language":"Vietnamese"},
    config=config
)
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I a Fullstack Software Engineer")],"language":"Vietnamese"},
    config=config
)
repsonse.content

'Chào mừng bạn, Thang! Rất tuyệt khi biết bạn là một Fullstack Software Engineer. \n\nBạn đang làm việc trên dự án gì thú vị hiện nay? \n'

In [24]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name and job?")], "language": "Vietnamese"},
    config=config,
)

In [25]:
response.content

'Tên của bạn là Thang và bạn là một Fullstack Software Engineer.  \n\nBạn muốn tôi nhớ thêm bất cứ thông tin gì về bạn không?  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [ ]:
from langchain_core.messages import SystemMessage,trim_messages
"""
If we are using max_tokens=70, the history will include all messages
"""
trimmer = trim_messages(
    max_tokens=45,        # Maximum tokens allowed in conversation
    strategy="last",      # Trim from end of conversation
    token_counter=model,  # Use model's tokenizer
    include_system=True,  # Preserve system messages
    allow_partial=False,  # Remove whole messages only
    start_on="human"      # Start at first human message
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/home/thangtran3112/gen-ai/langchain/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

# When invoked:
# 1. Gets messages using itemgetter
# 2. Passes through trimmer
# 3. Assigns result back to 'messages' key
# 4. Returns modified dictionary
messages_property_trimmer = RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)


"""
Using chain to trim the messages automatically
"""
chain=(messages_property_trimmer | prompt | model)

# 1st message in history has been trimmed
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I have no memory of past conversations and don't know your preferences.  \n\nWhat's your favorite ice cream flavor? 😊🍦\n"

In [32]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked what 2 + 2 equals.  \n\n\n\nIs there anything else I can help you with? 😊\n'

In [33]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [34]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊  \n\n"

In [35]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  \n\nWhat's the problem?  \n\n"